In [2]:
# import dependencies
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
from cytoolz import compose
import operator as op

import pandas as pd
import seaborn as sns
import numpy as np
np.object = object
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell

/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/seaborn/_statistics.py:32: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  from scipy.stats import gaussian_kde


In [3]:
# settings

# Set maximum number of jobs
sc.settings.njobs = 10

In [4]:
BASE_FOLDER = ""

EXP_MTX_QC_TPM_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.qc.tpm.csv')
METADATA_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.metadata.csv')
ANNDATA_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.h5ad')

AUCELL_MTX_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.auc.csv')
REGULONS_DAT_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.regulons.dat')
LOOM_FNAME = os.path.join(BASE_FOLDER, 'lilacs_tcr.loom')

In [5]:
adata = sc.read_h5ad("aa_aligned_adata_sc.h5ad")

In [6]:
adata.to_df().to_csv(EXP_MTX_QC_TPM_FNAME)

In [7]:
def derive_regulons(code, folder):
    # Load enriched motifs.
    motifs = load_motifs(os.path.join(folder, '{}.motifs.csv'.format(code)))
    #print(motifs)
    motifs.columns = motifs.columns.droplevel(0)

    def contains(*elems):
        def f(context):
            return any(elem in context for elem in elems)
        return f

    # For the creation of regulons we only keep the 10-species databases and the activating modules. We also remove the
    # enriched motifs for the modules that were created using the method 'weight>50.0%' (because these modules are not part
    # of the default settings of modules_from_adjacencies anymore.
    motifs = motifs[
        np.fromiter(map(compose(op.not_, contains('weight>50.0%')), motifs.Context), dtype=bool) & \
        np.fromiter(map(contains('hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings', 
                                 'hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings', 
                                 'hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings',
                                 'hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings'), motifs.Context), dtype=bool) & \
        np.fromiter(map(contains('activating'), motifs.Context), dtype=bool)]

    # We build regulons only using enriched motifs with a NES of 3.0 or higher; we take only directly annotated TFs or TF annotated
    # for an orthologous gene into account; and we only keep regulons with at least 10 genes.
   
    #regulons = list(filter(lambda r: len(r) >= 10, df2regulons(motifs[(motifs['NES'] >= 3.0) 
    #                                                                  & ((motifs['Annotation'] == 'gene is directly annotated')
    #                                                                    | (motifs['Annotation'].str.startswith('gene is orthologous to')
    #                                                                       & motifs['Annotation'].str.endswith('which is directly annotated for motif')))
    #                                                                 ])))
    
    #regulons = list(filter(lambda r: len(r) >= 7, df2regulons(motifs[(motifs['NES'] >= 3.0)])))
    regulons = list(filter(lambda r: len(r) >= 5, df2regulons(motifs[(motifs['NES'] >= 3.0)])))
    
    # Rename regulons, i.e. remove suffix.
    regulons = list(map(lambda r: r.rename(r.transcription_factor), regulons))

    # Pickle these regulons.
    with open(os.path.join(folder, '{}.regulons.dat'.format(code)), 'wb') as f:
        pickle.dump(regulons, f)

In [8]:
derive_regulons('lilacs_tcr', BASE_FOLDER)

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [9]:
with open(REGULONS_DAT_FNAME, 'rb') as f:
    regulons = pickle.load(f)

In [10]:
exp_mtx = pd.read_csv("lilacs_tcr.qc.tpm.csv", index_col=0)
auc_mtx = aucell(exp_mtx, regulons, num_workers=20)

/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/numba/__init__.py:48: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  import scipy
/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/numba/__init__.py:48: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  import scipy
/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/numba/__init__.py:48: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  import scipy
/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/numba/__init__.py:48: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  import scipy
/opt/anaconda3/envs/scenic/lib/python3.10/site-packages/numba/__init__.py:48: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected v

In [11]:
auc_mtx.to_csv(AUCELL_MTX_FNAME)

In [12]:
add_scenic_metadata(adata, auc_mtx, regulons)

AnnData object with n_obs × n_vars = 41050 × 1397
    obs: 'sampleid', 'sampleid_study', 'timepoint', 'treatment', 'study_id', 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'initial_clustering', 'fine_clustering', 'treatment_timepoint', 'sex', 'age', 'peak_trop', 'BNP', 'on_treatment_CRP', 'AEs', 'treatment_group_1', 'treatment_group_2', 'Lymph', 'CD4', 'CD8', 'Treg', 'sample_id', 'is_cell', 'high_confidence', 'multi_chain', 'extra_chains', 'IR_VJ_1_c_call', 'IR_VJ_2_c_call', 'IR_VDJ_1_c_call', 'IR_VDJ_2_c_call', 'IR_VJ_1_consensus_count', 'IR_VJ_2_consensus_count', 'IR_VDJ_1_consensus_count', 'IR_VDJ_2_consensus_count', 'IR_VJ_1_d_call', 'IR_VJ_2_d_call', 'IR_VDJ_1_d_call', 'IR_VDJ_2_d_call', 'IR_VJ_1_duplicate_count', 'IR_VJ_2_duplicate_count', 'IR_VDJ_1_duplicate_count', 'IR_VDJ_2_duplicate_count', 'IR_VJ_1_j_call', 'IR_VJ_2_j_call', 'IR_VDJ_1_j_call', '